Install the required dependencies

In [3]:
!pip install lancedb langchain openai tiktoken pypdf tantivy sentence-transformers langchain-community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00


# download your choice pdf file & upload here

 https://github.com/akashAD98/dummy_data/blob/main/microsoft_annual_report_2022.pdf

In [4]:
import os
import time
import lancedb
import getpass
import pandas as pd
from lancedb.embeddings import EmbeddingFunctionRegistry
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import LanceDB


# Set the OPENAI_API_KEY environment variable
os.environ["OPENAI_API_KEY"] = "sk-proj-...."

In [5]:
def word_wrap(string, n_chars=72):
    # Wrap a string at the next space after n_chars
    if len(string) < n_chars:
        return string
    else:
        return (
            string[:n_chars].rsplit(" ", 1)[0]
            + "\n"
            + word_wrap(string[len(string[:n_chars].rsplit(" ", 1)[0]) + 1 :], n_chars)
        )

We are using langchain fro loading the pdf file &  doing prprocessing

In [6]:
# your pdf file
loader = PyPDFLoader("/content/microsoft_annual_report_2022.pdf")
documents = loader.load_and_split()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docsn = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

docsearch = LanceDB.from_documents(docsn, embeddings)

query = "What were the most important factors that contributed to increases in revenue?"
docsnew = docsearch.similarity_search(query)
print(docsnew[0].page_content)

<ipython-input-6-393fdf8f6934>:9: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


35   
Reportable Segments  
Fiscal Year 2022 Compared with Fiscal Year 2021  
Productivity and Business Processes  
Revenue increased $9.4  billion or 18%.  
• Office Commercial products and cloud services revenue increased $4.4  billion or 13%. Office 365 
Commercial revenue grew 18% driven by seat growth of 14%, with continued momentum in small and 
medium business and frontline worker offerings, as well as growth in revenue per user. Office Commercial 
products revenue declined 22% driven by continued customer shift to cloud offerings.  
• Office Consumer products and cloud services revenue increased $641  million or 11% driven by Microsoft 365 
Consumer subscription revenue. Microsoft 365 Consumer subscribers grew 15% to 59.7  million.  
• LinkedIn revenue increased $3.5  billion or 34% driven by a strong job market in our Talent Solutions business 
and advertising demand in our Marketing Solutions business.  
• Dynamics products and cloud services revenue increased 25% driven by D

### Query expansion

In [7]:
# we are using openai for generating query
import os
import openai
from openai import OpenAI

openai.api_key = os.environ["OPENAI_API_KEY"]
openai_client = OpenAI()


def augment_query_generated(query, model="gpt-3.5-turbo"):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Provide an example answer to the given question, that might be found in a document like an annual report. ",
        },
        {"role": "user", "content": query},
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

### its generating  query & hypothetica answer

In [8]:
original_query = (
    "What were the most important factors that contributed to increases in revenue?"
)
hypothetical_answer = augment_query_generated(original_query)
# we are combining our orignal query + hypothetical_answer
joint_query = f"{original_query} {hypothetical_answer}"
print(word_wrap(joint_query))

What were the most important factors that contributed to increases in
revenue? In the current fiscal year, our company experienced
significant revenue growth due to a combination of factors. Firstly, we
successfully launched several new products that were well-received in
the market, leading to increased sales and market share. Additionally,
our strategic marketing initiatives helped to enhance brand awareness
and attract new customers. Furthermore, we expanded our distribution
network, allowing us to reach a wider customer base and drive higher
sales volume. Lastly, cost control measures and operational
efficiencies enabled us to improve profit margins and maximize revenue
generation. Overall, the successful execution of these strategies
significantly contributed to the increase in revenue during the year.


In [9]:
# we are passing the  joint_query & fining the  similar docs for this query
retrieved_documents = docsearch.similarity_search(joint_query, k=4)
print(retrieved_documents)

[Document(metadata={'page': 36, 'source': '/content/microsoft_annual_report_2022.pdf'}, page_content='35   \nReportable Segments  \nFiscal Year 2022 Compared with Fiscal Year 2021  \nProductivity and Business Processes  \nRevenue increased $9.4  billion or 18%.  \n• Office Commercial products and cloud services revenue increased $4.4  billion or 13%. Office 365 \nCommercial revenue grew 18% driven by seat growth of 14%, with continued momentum in small and \nmedium business and frontline worker offerings, as well as growth in revenue per user. Office Commercial \nproducts revenue declined 22% driven by continued customer shift to cloud offerings.  \n• Office Consumer products and cloud services revenue increased $641  million or 11% driven by Microsoft 365 \nConsumer subscription revenue. Microsoft 365 Consumer subscribers grew 15% to 59.7  million.  \n• LinkedIn revenue increased $3.5  billion or 34% driven by a strong job market in our Talent Solutions business \nand advertising dema

In [10]:
# sample of retrieved_documents
print(retrieved_documents[0].page_content)

35   
Reportable Segments  
Fiscal Year 2022 Compared with Fiscal Year 2021  
Productivity and Business Processes  
Revenue increased $9.4  billion or 18%.  
• Office Commercial products and cloud services revenue increased $4.4  billion or 13%. Office 365 
Commercial revenue grew 18% driven by seat growth of 14%, with continued momentum in small and 
medium business and frontline worker offerings, as well as growth in revenue per user. Office Commercial 
products revenue declined 22% driven by continued customer shift to cloud offerings.  
• Office Consumer products and cloud services revenue increased $641  million or 11% driven by Microsoft 365 
Consumer subscription revenue. Microsoft 365 Consumer subscribers grew 15% to 59.7  million.  
• LinkedIn revenue increased $3.5  billion or 34% driven by a strong job market in our Talent Solutions business 
and advertising demand in our Marketing Solutions business.  
• Dynamics products and cloud services revenue increased 25% driven by D

#Reranker model

we are using landb for colbertv2 implementation
COlbertv2 is reranker method helpful for improving the ranking of results its returning top ranking
results as the most relevant to your particular query.

for more technical details please check the blog :

In [11]:
db = lancedb.connect("/tmp/db")
registry = EmbeddingFunctionRegistry.get_instance()
func = registry.get("openai").create()


class Words(LanceModel):
    text: str = func.SourceField()
    vector: Vector(func.ndims()) = func.VectorField()


table = db.create_table("wordsllm", schema=Words, mode="overwrite")

# data from retriver
formatted_data = [{"text": doc.page_content} for doc in retrieved_documents]


# ingest docs with auto-vectorization
table.add(formatted_data)
# Create the FTS index on the 'text' field
table.create_fts_index(["text"], replace=True)


# normal search
query = "technologies and business models"
actual = table.search(query).limit(1).to_pydantic(Words)[0]
print(actual.text)

31   
Industry Trends  
Our industry is dynamic and highly competitive, with frequent changes in both technologies and business models. Each 
industry shift is an opportunity to conceive new products, new technologies, or new ideas that can further transform the 
industry and our business. At Microsoft, we push the boundaries of what is possible through a broad range of research 
and development activities that seek to identify and address the changing demands of customers and users, industry 
trends, and competitive forces.  
Economic Conditions, Challenges, and Risks  
The markets for software, devices, and cloud -based services are dynamic and highly competitive. Our competitors are 
developing new software and devices, while also deploying competing cloud -based services for consumers and 
businesses. The devices and form factors customers prefer evolve rapidly, and influence how users access services in 
the cloud, and in some cases, the user’s choice of which suite of cloud -base

#Colbertv2 reranker model

In [16]:
!pip install rerankers

In [17]:
from lancedb.rerankers import ColbertReranker

reranker_colbert = ColbertReranker()
results_colbert = (
    table.search("technologies and business models", query_type="hybrid")
    .rerank(reranker=reranker_colbert)
    .to_pandas()
)

Loading ColBERTRanker model colbert-ir/colbertv2.0
No device set
Using device cpu
No dtype set
Device set to `cpu`, setting dtype to `float32`
Using dtype torch.float32
Loading model colbert-ir/colbertv2.0, this might take a while...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Linear Dim set to: {linear_dim} for downcasting


In [18]:
results_colbert

,text,vector,_relevance_score
0,31 \nIndustry Trends \nOur industry is dyna...,"[0.0092269145, -0.020611297, -0.0066352366, -0...",1.245123
1,33 Dynamics products and cloud services revenu...,"[-0.0026319, -0.023140918, -0.009075517, -0.01...",0.705481
2,35 \nReportable Segments \nFiscal Year 2022...,"[-0.0043231365, -0.022661446, -0.014712347, -0...",0.510181
3,34 net income tax benefit related to an India ...,"[-0.014394735, -0.014434795, 0.0076446994, -0....",0.445338


Lets try the CrossEncoder for getting relavant docus from it

## cross-encoder reranker model

Re-ranking is a way to order results and score
them according to their relevancy to a particular
query.
So let's take a look at how this works underneath.
In re-ranking, after you retrieve results for
a particular query, you pass these results along with
your query to a re-ranking model. This allows you to re-rank the output
so the most relevant results have the highest
rank. Another way to think about this is
your re-ranking model scores each of the results conditioned on the
query, and those with the highest score are the most
relevant.
Then you can just select the top ranking
results as the most relevant to your particular query.
So let's take a look at how to do this in practice

In [19]:
import numpy as np

# cross encoder reranker
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

# Extract text content from Document objects and convert to strings
document_texts = [doc.page_content for doc in retrieved_documents]
query_text = (
    "What were the most important factors that contributed to increases in revenue?"
)
# Create pairs as strings
pairs = [[query_text, doc_text] for doc_text in document_texts]
# Predict scores for pairs
scores = cross_encoder.predict(pairs)
# Print scores
print("Scores:")
for score in scores:
    print(score)


print("New Ordering:")
for o in np.argsort(scores)[::-1]:
    print(o + 1)

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Scores:
-4.676173
-4.906811
-9.046174
-3.4381006
New Ordering:
4
1
2
3


In [20]:
pairs

[['What were the most important factors that contributed to increases in revenue?',
  '35   \nReportable Segments  \nFiscal Year 2022 Compared with Fiscal Year 2021  \nProductivity and Business Processes  \nRevenue increased $9.4  billion or 18%.  \n• Office Commercial products and cloud services revenue increased $4.4  billion or 13%. Office 365 \nCommercial revenue grew 18% driven by seat growth of 14%, with continued momentum in small and \nmedium business and frontline worker offerings, as well as growth in revenue per user. Office Commercial \nproducts revenue declined 22% driven by continued customer shift to cloud offerings.  \n• Office Consumer products and cloud services revenue increased $641  million or 11% driven by Microsoft 365 \nConsumer subscription revenue. Microsoft 365 Consumer subscribers grew 15% to 59.7  million.  \n• LinkedIn revenue increased $3.5  billion or 34% driven by a strong job market in our Talent Solutions business \nand advertising demand in our Marke

Now you can use this reranked documents & pass it to llm & get relavant results. please check our other techical blogs also where we have explained other important methods of how you can improve your rag

#FlashRanker

Ultra-lite & Super-fast Python library to add re-ranking to your existing search & retrieval pipelines. It is based on SoTA cross-encoders.

In [21]:
!pip install flashrank

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.2 MB/s eta 0:00:00


In [22]:
# Metadata is optional, Id can be your DB ids from your retrieval stage or simple numeric indices.
from flashrank import Ranker, RerankRequest

# query = "How to speedup LLMs?"

query = "What were the most important factors that contributed to increases in revenue?"

ranker = Ranker(model_name="ms-marco-MiniLM-L-12-v2", cache_dir="/opt")

rerankrequest = RerankRequest(query=query, passages=formatted_data)
results = ranker.rerank(rerankrequest)
print(results)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:00<00:00, 106MiB/s]


[{'text': '35   \nReportable Segments  \nFiscal Year 2022 Compared with Fiscal Year 2021  \nProductivity and Business Processes  \nRevenue increased $9.4  billion or 18%.  \n• Office Commercial products and cloud services revenue increased $4.4  billion or 13%. Office 365 \nCommercial revenue grew 18% driven by seat growth of 14%, with continued momentum in small and \nmedium business and frontline worker offerings, as well as growth in revenue per user. Office Commercial \nproducts revenue declined 22% driven by continued customer shift to cloud offerings.  \n• Office Consumer products and cloud services revenue increased $641  million or 11% driven by Microsoft 365 \nConsumer subscription revenue. Microsoft 365 Consumer subscribers grew 15% to 59.7  million.  \n• LinkedIn revenue increased $3.5  billion or 34% driven by a strong job market in our Talent Solutions business \nand advertising demand in our Marketing Solutions business.  \n• Dynamics products and cloud services revenue i

**This is how you can use this methods & pass this data to llm & get better results fro your RAG applications**